In [67]:
!pip install roboflow

import tensorflow as tf

from roboflow import Roboflow

rf = Roboflow(api_key="zJBGzdUCic7EvF42E0hk")  # Replace with your API Key
project = rf.workspace("uveitis").project("eye-diseases-zncfk")
dataset = project.version("3").download("tfrecord")

loading Roboflow workspace...
loading Roboflow project...


In [68]:
def parse_tfrecord(example_proto):
    feature_description = {
        'image/encoded': tf.io.FixedLenFeature([], tf.string),
        'image/height': tf.io.FixedLenFeature([], tf.int64),
        'image/width': tf.io.FixedLenFeature([], tf.int64),
        'image/object/class/text': tf.io.VarLenFeature(tf.string)  # Labels
    }
    parsed_example = tf.io.parse_single_example(example_proto, feature_description)

    # Decode the image
    image = tf.io.decode_jpeg(parsed_example['image/encoded'], channels=3)
    image = tf.image.resize(image, (224, 224))  # Resize to match the model input
    image = image / 255.0  # Normalize pixel values

    # Convert sparse labels to dense tensor
    label = tf.sparse.to_dense(parsed_example['image/object/class/text'])

    # Convert text labels to integer labels (you need a mapping)
    label_map = {'class_1': 0, 'class_2': 1, 'class_3': 2}  # Example class mapping
    label = tf.map_fn(lambda x: label_map[x.numpy().decode('utf-8')], label, dtype=tf.int32)

    # Convert to one-hot encoding (adjust number of classes)
    num_classes = len(label_map)
    label = tf.one_hot(label, num_classes)

    return image, label


In [77]:
dataset_path = "/Eye-Diseases-3/train.tfrecord"
raw_dataset = tf.data.TFRecordDataset(dataset_path)
parsed_dataset = raw_dataset.map(parse_tfrecord)

for image, label in parsed_dataset.take(1):
    print("Image shape:", image.shape)
    print("Label:", label.numpy())

NotFoundError: {{function_node __wrapped__IteratorGetNext_output_types_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} NewRandomAccessFile failed to Create/Open: /Eye-Diseases-3/train.tfrecord : The system cannot find the path specified.
; No such process [Op:IteratorGetNext] name: 

In [75]:
train_dataset = parsed_dataset.batch(32).repeat()

Epoch 1/10


TypeError: Failed to convert elements of SparseTensor(indices=Tensor("data_1:0", shape=(None, 2), dtype=int64), values=Tensor("compile_loss/categorical_crossentropy/Cast:0", shape=(None,), dtype=float32), dense_shape=Tensor("data_3:0", shape=(2,), dtype=int64)) to Tensor. Consider casting elements to a supported type. See https://www.tensorflow.org/api_docs/python/tf/dtypes for supported TF dtypes.

In [ ]:
model.fit(train_dataset, epochs=10, steps_per_epoch=100)